## Network Setup

1. Download Multimodal network dataset from [https://gis.utah.gov/data/transportation/street-network-analysis/]
2. Download Wasatch Front TAZs from WFRC [https://data.wfrc.org/datasets/access-to-opportunities-work-related-taz-based/] (https://data.wfrc.org/datasets/access-to-opportunities-work-related-taz-based/)
2. Copy the Multimodal network dataset and delete the layers that will be replaced with clipped versions
3. Clip the NetworkDataset>BikePedAuto and NetworkDataset>Network_Dataset_ND_Junctions layers using 15 mile buffer around the Wasatch Front TAZ layer
4. Create and fill RoadClass attribute to a usable hierarchy value
5. Join the TDM data for speeds


** Note: sometimes sequential execution of the cells below will produce a "Cannot acquire a lock" error. If this happens, just re-run the cell. Esri weirdness! **


## TAZ Centroids

Calculate a centroid point for each TAZ from the TAZ shapefile. If you have an "Advanced" license of ArcGIS Pro you can do this with the "Feature to Point" tool. Otherwise, just fire up QGIS, open the TAZ layer, Go to the menu -> Vector -> Geometry tools -> polygon centroid and create a centroid point layer.

# Network Setup

Vertical Elevation - the current dataset does not contain the necessary attributes to establish vertical connectivity. Elevation is not used.

Hierarchy - oddly, adding a heirarchy for routing seems to worsen performance. Hierarchy is not used.

In [1]:
import arcpy
import os

# Set the XYResolution environment to a linear unit
arcpy.env.XYResolution = "0.01 Meters"
arcpy.env.XYTolerance = "0.1 Meters"

# set base path - jupyter or arcgis
try:
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    print("base path must be explicitly set when running in arcgis pro")
    base_path = r"c:\wfrc\ato" # update this
except OSError:
    base_path = "."
    
arcpy.CheckOutExtension("network")

source_mm_network_dataset = os.path.join(base_path, r"shp\MM_NetworkDataset_09022021.gdb\NetworkDataset")
tdm_shp_path = os.path.join(base_path, r"shp\Master_Segs_withFactors_20200504_BY2019SegSummary\Master_Segs_withFactors_20200504_BY2019SegSummary.shp")

base_gdb = os.path.join(base_path, "nd.gdb")
if not os.path.isdir(base_gdb):
    arcpy.management.CreateFileGDB(base_path, "nd")

arcpy.env.workspace = base_gdb

In [2]:
# Copy NetworkDataset to our working GDB
arcpy.management.Copy(
    source_mm_network_dataset, 
    os.path.join(base_gdb, "NetworkDataset")
)

<Result '.\\nd.gdb\\NetworkDataset'>

In [3]:
# delete existing network
arcpy.management.Delete(os.path.join(base_gdb, r"NetworkDataset\NetworkDataset_ND"))

<Result 'true'>

In [4]:
arcpy.management.MakeFeatureLayer(
    os.path.join(base_gdb, r"NetworkDataset\BikePedAuto"), 
    "BPA"
)

<Result 'BPA'>

In [5]:
# add hierarchy to clipped BikePedAuto
arcpy.management.AddField(
    "BPA", 
    "hierarchy", 
    "SHORT"
)

<Result 'BPA'>

In [6]:
# if this fails due to a "lock" error try running again
# note, this field was previously RoadClass and is now CartoCode
expression = "getClass(!CartoCode!)"

codeblock = """
hierarchy = {
	'1 Interstates': 1,
	'2 US Highways, Separated': 1,
	'3 - Paved Shared Use': 3,
	'3 US Highways, Unseparated': 1,
	'4 Major State Highways, Separated': 1,
	'5 Major State Highways, Unseparated': 1,
	'6 Other State Highways (Institutional)': 1,
	'7 Ramps, Collectors': 2,
	'8 - Bridge, Tunnel': 2,
	'8 Major Local Roads, Paved': 2,
	'9 - Link': 3,
	'9 Major Local Roads, Not Paved': 3,
	'10 Other Federal Aid Eligible Local Roads': 2,
	'11 Other Local, Neighborhood, Rural Roads': 3,
	'12 Other': 3,
	'13 Non-road feature': None,
	'14 Driveway': None,
	'15 Proposed': None,
	'17 Service Access Roads': None,
	'99 - UDOT FAE Calibration (Non-Road Feature)': None
}
def getClass(rc):
    return 3 if rc is None else hierarchy[rc]
"""

# Execute CalculateField 
arcpy.management.CalculateField("BPA", "hierarchy", expression, "PYTHON3", codeblock)

<Result 'BPA'>

In [7]:
# Select non-road features
arcpy.management.SelectLayerByAttribute(
    "BPA", 
    "NEW_SELECTION", 
    "hierarchy IS NULL"
)

id,value
0,a Layer object
1,869


In [8]:
# Create a 5 mile buffer
arcpy.analysis.Buffer(
    in_features = os.path.join(base_path, r"shp\taz_wfrc.gdb\ATO"),
    out_feature_class = os.path.join(base_gdb, "taz_buffer"),
    buffer_distance_or_field = "5 Miles",
    dissolve_option = "ALL"
)

<Result '.\\nd.gdb\\taz_buffer'>

In [9]:
arcpy.management.DeleteFeatures("BPA")
# sometimes this fails - removing from map and adding again seems to help

<Result 'BPA'>

In [10]:
arcpy.management.Delete(os.path.join(base_gdb, "taz_buffer"))

<Result 'true'>

# Integrate

To fix some issues where lines don't connect, highlight all features where AutoRoute == Y and then use the Pairwise Integrate tool on BikePedAuto. This will create a vertex (connection) at all line intersections. To avoid connecting overpasses, we unselect 

In [11]:
# Select non-road features
arcpy.management.SelectLayerByAttribute(
    "BPA", 
    "NEW_SELECTION", 
    "AutoNetwork = 'Y' AND VERT_LEVEL = '0'"
    # remove bridges and tunnels
)
# pairwise integrate
arcpy.analysis.PairwiseIntegrate("BikePedAuto", None)

<Result '.\\nd.gdb\\BikePedAuto'>

In [12]:
arcpy.management.SelectLayerByAttribute("BPA", "CLEAR_SELECTION")

id,value
0,a Layer object
1,-1


### Join in Speed Data
WFRC has provided free flow and peak hour speed data from their TDM. We will join this to the network dataset for later use when roughly estimating the short-term improvement in automobile mobility associated with roadway widening.

In [13]:
arcpy.conversion.FeatureClassToFeatureClass(
    tdm_shp_path,
    base_gdb,
    "TDM"
)

<Result '.\\nd.gdb\\TDM'>

In [14]:
# Create and calculate PK_SPD field based on minimum (peak time) speed
arcpy.management.CalculateField(
    "TDM", 
    "PK_SPD", 
    "min(!AM_SPD!,!MD_SPD!,!PM_SPD!,!EV_SPD!)", 
    "PYTHON3", '', "DOUBLE"
)

<Result '.\\nd.gdb\\TDM'>

In [15]:
# Spatially join TDM travel times to MM network:
field_spec = """Name "Name" true true false 50 Text 0 0,First,#,BikePedAuto,Name,0,50;
Oneway "Oneway" true true false 2 Text 0 0,First,#,BikePedAuto,Oneway,0,2;
Speed "Speed" true true false 2 Short 0 0,First,#,BikePedAuto,Speed,-1,-1;
AutoNetwork "AutoNetork" true true false 1 Text 0 0,First,#,BikePedAuto,AutoNetwork,0,1;
BikeNetwork "BikeNetwork" true true false 1 Text 0 0,First,#,BikePedAuto,BikeNetwork,0,1;
PedNetwork "PedNetwork" true true false 1 Text 0 0,First,#,BikePedAuto,PedNetwork,0,1;
SourceData "SourceData" true true false 15 Text 0 0,First,#,BikePedAuto,SourceData,0,15;
DriveTime "DriveTime" true true false 8 Double 0 0,First,#,BikePedAuto,DriveTime,-1,-1;
BikeTime "BikeTime" true true false 8 Double 0 0,First,#,BikePedAuto,BikeTime,-1,-1;
PedestrianTime "PedestrianTime" true true false 8 Double 0 0,First,#,BikePedAuto,PedestrianTime,-1,-1;
Length_Miles "Length_Miles" true true false 8 Double 0 0,First,#,BikePedAuto,Length_Miles,-1,-1;
ConnectorNetwork "ConnectorNetwork" true true false 1 Text 0 0,First,#,BikePedAuto,ConnectorNetwork,0,1;
CartoCode "CartoCode" true true false 50 Text 0 0,First,#,BikePedAuto,CartoCode,0,50;
AADT "AADT" true true false 4 Long 0 0,First,#,BikePedAuto,AADT,-1,-1;
AADT_YR "AADT_YR" true true false 4 Text 0 0,First,#,BikePedAuto,AADT_YR,0,4;
BIKE_L "BIKE_L" true true false 4 Text 0 0,First,#,BikePedAuto,BIKE_L,0,4;
BIKE_R "BIKE_R" true true false 4 Text 0 0,First,#,BikePedAuto,BIKE_R,0,4;
VERT_LEVEL "VERT_LEVEL" true true false 25 Text 0 0,First,#,BikePedAuto,VERT_LEVEL,0,25;
hierarchy "hierarchy" true true false 2 Short 0 0,First,#,BikePedAuto,hierarchy,-1,-1;
FF_SPD "FF_SPD" true true false 8 Double 0 0,First,#,TDM,FF_SPD,-1,-1;
PK_SPD "PK_SPD" true true false 8 Double 0 0,First,#,TDM,PK_SPD,-1,-1"""

arcpy.analysis.SpatialJoin(
    "BPA", 
    "TDM", 
    os.path.join(base_gdb, "TDM_SpatialJoin"), 
    "JOIN_ONE_TO_ONE", 
    "KEEP_ALL", 
    field_spec, 
    "HAVE_THEIR_CENTER_IN", # "SHARE_A_LINE_SEGMENT_WITH"
)

<Result '.\\nd.gdb\\TDM_SpatialJoin'>

In [16]:
# copy attributes back to the BikePedAuto
arcpy.management.JoinField(
    "BPA", 
    "OBJECTID", 
    "TDM_SpatialJoin", 
    "TARGET_FID", 
    "FF_SPD;PK_SPD"
)

<Result 'BPA'>

In [17]:
# Calculate drive times based on peak hour TDM speeds
codeblock = """
def getSpeed(spd, len):
    return None if spd is None else float(60*len/spd)
"""

arcpy.management.CalculateField(
    "BPA", 
    "DriveTime_Peak", 
    "getSpeed(!PK_SPD!, !Length_Miles!)", 
    "PYTHON3", codeblock, "DOUBLE"
)
arcpy.management.CalculateField(
    "BPA", 
    "DriveTime_FF", 
    "getSpeed(!FF_SPD!, !Length_Miles!)", 
    "PYTHON3", codeblock, "DOUBLE"
)

<Result 'BPA'>

In [18]:
# Update Drive Time based on TDM peak value when set/exists
arcpy.CalculateField_management(
    "BPA", 
    "DriveTime", 
    "!DriveTime! if !DriveTime_Peak! is None else !DriveTime_Peak!", 
    "PYTHON3", None, "DOUBLE")

<Result 'BPA'>

In [19]:
# delete existing BikePedAuto layer
arcpy.management.Delete("TDM_SpatialJoin")
arcpy.management.Delete("TDM")

<Result 'true'>

In [21]:
# create template from MM network
arcpy.na.CreateTemplateFromNetworkDataset(
    os.path.join(source_mm_network_dataset, "NetworkDataset_ND"),
    os.path.join(base_path, "template.xml")
)

<Result '.\\template.xml'>

In [22]:
# create network dataset from template
arcpy.na.CreateNetworkDatasetFromTemplate(
    os.path.join(base_path, "template.xml"),                           
    os.path.join(base_gdb, "NetworkDataset")
)

<Result '.\\nd.gdb\\NetworkDataset\\NetworkDataset_ND'>

In [1]:
# finally, build the dataset
arcpy.na.BuildNetwork(os.path.join(base_gdb, r"NetworkDataset\NetworkDataset_ND"))

NameError: name 'os' is not defined

In [ ]:
# now open arcgis pro and define a "driving" mode